In [59]:
library(dplyr)
library(tidyr)
library(stringr)
library(geohash)

In [60]:
library(downloader)
# Blight Violations
dbvurl <- "https://d18ky98rnyall9.cloudfront.net/_97bd1c1e5df9537bb13398c9898deed7_detroit-blight-violations.csv?Expires=1484524800&Signature=hlHM02mABfvFUvErGgrV3u31Nr8GO6cwNPX1Z0wzrdY5o8q7faWEsmeoOcGE5p43iH3-WEtliSCduHMa~nrVcxeZpM-vvNhNXIU3D8m~0yBqrjbCpTeVdaC0IA-RgKNN1NfgMHoeSjSItNmjXcVfFq3U5X~xbNuv0DI4No8Qe80_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DBVFileName <- "detroit-blight-violations.csv"
if (!file.exists(DBVFileName)) 
    download(dbvurl, DBVFileName)
detBlightViol <- read.csv("detroit-blight-violations.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )   
    
# Detroit Demolition Permits 
ddpurl <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-demolition-permits.tsv?Expires=1487116800&Signature=A-ZtSkr8DZMpetuP4NNGPpz327wizHe9iw2SUTvlU5-eOJ1eTgnIflqgmfnaJJ-ultJMdawSbRD-tp5vtc45CnB1E9nMGwO-mFrca3z8bz3HKL09YgJEU2bM571hYeULyucNTcjejsQMmfZno8N2Qs9AwDigjlD1Ixy-pnyk2Ns_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DDPFileName <- "detroit-demolition-permits.tsv"
if (!file.exists(DDPFileName)) 
    download(ddpurl, DDPFileName)
detDemolitionPermit <- read.delim("detroit-demolition-permits.tsv", header=TRUE, allowEscapes=FALSE, sep="\t", na.strings="", comment.char="")

    
# Detroit 311 calls
d311url <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv?Expires=1487116800&Signature=i9Aq7nh9-0FvxpRKJLXXxN~XA~vVQ-Nk5AndCHEaoBY5vAf1CT8lt9xnVXwL5sbcMGD3LDIc~6LnB8daTqFgJFgKki86q885HsgiLH2-GnZy5zVmSEtkv0bPmMKwPbs10SNgNl7td~8CUb3MhmpUiQr1KjbeYTQYYZWUzJGZGNI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
D311FileName <- "detroit-311.csv"
if (!file.exists(D311FileName)) 
    download(d311url, D311FileName)
det311 <- read.csv("detroit-311.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )  

# Detroit Crime 
dcrurl <- "https://d18ky98rnyall9.cloudfront.net/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv?Expires=1487116800&Signature=i9Aq7nh9-0FvxpRKJLXXxN~XA~vVQ-Nk5AndCHEaoBY5vAf1CT8lt9xnVXwL5sbcMGD3LDIc~6LnB8daTqFgJFgKki86q885HsgiLH2-GnZy5zVmSEtkv0bPmMKwPbs10SNgNl7td~8CUb3MhmpUiQr1KjbeYTQYYZWUzJGZGNI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
DCRFileName <- "detroit-crime.csv"
if (!file.exists(DCRFileName)) 
    download(dcrurl, DCRFileName)
detCrime <- read.csv("detroit-crime.csv", stringsAsFactors = FALSE,
                  strip.white = TRUE, na.strings ='' )    
       

In [61]:
#dplyr mutate to extract (LAT,LNG) from ViolationAddress
detBlightViol <- detBlightViol %>%
    mutate(latlng = gsub(".*\\n", "", ViolationAddress)) %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    filter(latlng != 'character(0)') %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    separate(latlng, c('LAT', 'LNG'), ', ')
#head(detBlightViol)

#dplyr mutate to extract (LAT,LNG) from site_location
detDemolitionPermit <- detDemolitionPermit %>%
    mutate(latlng = gsub(".*\\n", "", site_location)) %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    filter(latlng != 'character(0)') %>%
    transform(latlng = gsub('[()]','', latlng)) %>%
    separate(latlng, c('LAT', 'LNG'), ', ')



Warning message:
"Too few values at 14 locations: 18, 23, 28, 88, 89, 94, 99, 144, 172, 189, 194, 206, 227, 335"

In [62]:
#dplyr filter to create gh_8 gh_7 and uid colums 
detBlightViol <- detBlightViol %>%
    mutate(gh_8 = gh_encode(as.numeric(LAT), as.numeric(LNG), 8),
           gh_7 = gh_encode(as.numeric(LAT), as.numeric(LNG), 7),
           uid = paste0('blightviol_',row_number()))

detDemolitionPermit <- detDemolitionPermit %>%
    mutate(gh_8 = gh_encode(as.numeric(LAT), as.numeric(LNG), 8),
           gh_7 = gh_encode(as.numeric(LAT), as.numeric(LNG), 7),
           uid = paste0('demolper',row_number()))

det311 <- det311 %>%
    mutate(gh_8 = gh_encode(as.numeric(lat), as.numeric(lng), 8),
           gh_7 = gh_encode(as.numeric(lat), as.numeric(lng), 7),
           uid = paste0('det311',row_number()))

detCrime <- detCrime %>%
    mutate(gh_8 = gh_encode(as.numeric(lat), as.numeric(lng), 8),
           gh_7 = gh_encode(as.numeric(lat), as.numeric(lng), 7),
           uid = paste0('detcrim',row_number()))

In [63]:
dbv_gh_8_grp <- detBlightViol %>%
    group_by(gh_8) %>%
    summarize(dbv_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(dbv_ngbr_8_ct))

ddp_gh_8_grp <- detDemolitionPermit %>%
    group_by(gh_8) %>%
    summarize(ddp_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(ddp_ngbr_8_ct))

d311_gh_8_grp <- det311 %>%
    group_by(gh_8) %>%
    summarize(d311_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(d311_ngbr_8_ct))

dcr_gh_8_grp <- detCrime %>%
    group_by(gh_8) %>%
    summarize(dcr_ngbr_8_ct = n()) %>%
    arrange(gh_8,desc(dcr_ngbr_8_ct))


In [64]:
dbv_gh_7_grp <- detBlightViol %>%
    group_by(gh_7) %>%
    summarize(dbv_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(dbv_ngbr_7_ct))

ddp_gh_7_grp <- detDemolitionPermit %>%
    group_by(gh_7) %>%
    summarize(ddp_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(ddp_ngbr_7_ct))

d311_gh_7_grp <- det311 %>%
    group_by(gh_7) %>%
    summarize(d311_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(d311_ngbr_7_ct))

dcr_gh_7_grp <- detCrime %>%
    group_by(gh_7) %>%
    summarize(dcr_ngbr_7_ct = n()) %>%
    arrange(gh_7,desc(dcr_ngbr_7_ct))


In [65]:
detBlightViolV <- left_join(detBlightViol, dbv_gh_8_grp, by = c("gh_8" = "gh_8"))
detBlightViolVD <- left_join(detBlightViolV, ddp_gh_8_grp, by = c("gh_8" = "gh_8"))
detBlightViolVD3 <- left_join(detBlightViolVD, d311_gh_8_grp, by = c("gh_8" = "gh_8"))
detBlightViolVD3C <- left_join(detBlightViolVD3, dcr_gh_8_grp, by = c("gh_8" = "gh_8"))

In [66]:
detBlightViolVD3C[is.na(detBlightViolVD3C)] <- 0
keep <- c("LAT","LNG","gh_8", "gh_7", "dbv_ngbr_8_ct", "ddp_ngbr_8_ct", "d311_ngbr_8_ct", "dcr_ngbr_8_ct",  "ViolationCategory", "CleanUpCost", "JudgmentAmt", "PaymentStatus", "FineAmt", "AdminFee", "LateFee", "AgencyName")
detBlightViolVD3C <- detBlightViolVD3C[keep]


detBlightViolVD3C$PaymentStatus[detBlightViolVD3C$PaymentStatus=="PAID IN FULL"] <- 1
detBlightViolVD3C$PaymentStatus[detBlightViolVD3C$PaymentStatus=="PARTIAL PAYMENT MADE"] <- 2
detBlightViolVD3C$PaymentStatus[detBlightViolVD3C$PaymentStatus=="NO PAYMENT APPLIED"] <- 3
detBlightViolVD3C$PaymentStatus[detBlightViolVD3C$PaymentStatus=="NO PAYMENT ON RECORD"] <- 4

detBlightViolVD3C$AgencyName[detBlightViolVD3C$AgencyName=="Department of Public Works"] <- 1
detBlightViolVD3C$AgencyName[detBlightViolVD3C$AgencyName=="Building and Safety Engineering Department"] <- 2
detBlightViolVD3C$AgencyName[detBlightViolVD3C$AgencyName=="Health Department"] <- 3
detBlightViolVD3C$AgencyName[detBlightViolVD3C$AgencyName=="Detroit Police Department"] <- 4



In [89]:
detBlightViolVD3C$CleanUpCost[detBlightViolVD3C$CleanUpCost != ""] <- as.numeric(sub('\\$','',(as.character(detBlightViolVD3C$CleanUpCost))))
detBlightViolVD3C$CleanUpCost[detBlightViolVD3C$CleanUpCost == ""] <- 0

detBlightViolVD3C$JudgmentAmt[detBlightViolVD3C$JudgmentAmt != ""] <- as.numeric(sub('\\$','',(as.character(detBlightViolVD3C$JudgmentAmt))))
detBlightViolVD3C$JudgmentAmt[detBlightViolVD3C$JudgmentAmt == ""] <- 0

detBlightViolVD3C$FineAmt[detBlightViolVD3C$FineAmt != ""] <- as.numeric(sub('\\$','',(as.character(detBlightViolVD3C$FineAmt))))
detBlightViolVD3C$FineAmt[detBlightViolVD3C$FineAmt == ""] <- 0

detBlightViolVD3C$AdminFee[detBlightViolVD3C$AdminFee != ""] <- as.numeric(sub('\\$','',(as.character(detBlightViolVD3C$AdminFee))))
detBlightViolVD3C$AdminFee[detBlightViolVD3C$AdminFee == ""] <- 0

detBlightViolVD3C$LateFee[detBlightViolVD3C$LateFee != ""] <- as.numeric(sub('\\$','',(as.character(detBlightViolVD3C$LateFee))))
detBlightViolVD3C$LateFee[detBlightViolVD3C$LateFee == ""] <- 0

detBlightViolVD3C$Demolished[detBlightViolVD3C$ddp_ngbr_8_ct >0 ] <- 1
detBlightViolVD3C$Demolished[detBlightViolVD3C$ddp_ngbr_8_ct <=0 ] <- 0

#detBlightViolVD3C$dbv_ngbr_8_ct[detBlightViolVD3C$dbv_ngbr_8_ct =="" ||  detBlightViolVD3C$dbv_ngbr_8_ct == NULL] <- 0
#detBlightViolVD3C$ddp_ngbr_8_ct[detBlightViolVD3C$ddp_ngbr_8_ct =="" ||  detBlightViolVD3C$ddp_ngbr_8_ct == NULL ] <- 0
#detBlightViolVD3C$d311_ngbr_8_ct[detBlightViolVD3C$d311_ngbr_8_ct =="" ||  detBlightViolVD3C$d311_ngbr_8_ct == NULL ] <- 0
#detBlightViolVD3C$dcr_ngbr_8_ct[detBlightViolVD3C$dcr_ngbr_8_ct =="" ||  detBlightViolVD3C$dcr_ngbr_8_ct == NULL ] <- 0

#detBlightViolVD3C$ViolationCategory[detBlightViolVD3C$ViolationCategory =="" || detBlightViolVD3C$ViolationCategory == NULL] <- 0
#detBlightViolVD3C$PaymentStatus[detBlightViolVD3C$PaymentStatus =="" || detBlightViolVD3C$PaymentStatus == NULL] <- 0
#detBlightViolVD3C$AgencyName[detBlightViolVD3C$AgencyName =="" || detBlightViolVD3C$AgencyName == NULL] <- 0
ifelse({tmpDetBlightViolVD3C <- detBlightViolVD3C; tmpDetBlightViolVD3C!=0}, tmpDetBlightViolVD3C[,] <- 1, tmpDetBlightViolVD3C[,] <- 0) 
head(detBlightViolVD3C)


LAT,LNG,gh_8,gh_7,dbv_ngbr_8_ct,ddp_ngbr_8_ct,d311_ngbr_8_ct,dcr_ngbr_8_ct,ViolationCategory,CleanUpCost,JudgmentAmt,PaymentStatus,FineAmt,AdminFee,LateFee,AgencyName,Demolished
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0
1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1
1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0


LAT,LNG,gh_8,gh_7,dbv_ngbr_8_ct,ddp_ngbr_8_ct,d311_ngbr_8_ct,dcr_ngbr_8_ct,ViolationCategory,CleanUpCost,JudgmentAmt,PaymentStatus,FineAmt,AdminFee,LateFee,AgencyName,Demolished
42.36318237000006,-83.09167672099994,dpsbvpyr,dpsbvpy,12,0,0,0,0,0,1680,1,1500,20,150,1,0
42.429390762000025,-83.22039357799997,dpsc6k0c,dpsc6k0,5,0,0,0,0,0,140,3,100,20,10,1,0
42.428707459000066,-83.22754809599996,dpsc65gt,dpsc65g,3,0,0,0,0,0,140,1,100,20,10,1,0
42.44169828400004,-83.14501821599998,dpsc7yr8,dpsc7yr,8,0,0,0,0,0,140,3,100,20,10,1,0
42.42031769500005,-83.14532817999998,dpsc7frk,dpsc7fr,1,0,0,0,0,0,140,1,100,20,10,1,0
42.41788868500004,-83.15363728399996,dpsc7ccq,dpsc7cc,4,0,0,0,0,0,140,3,100,20,10,1,0


In [90]:
library(sqldf)
sqldf("select count(*) from 'detBlightViolVD3C' where ddp_ngbr_8_ct >0")
#sqldf("select count(*) from 'detBlightViol'")
# ddp_ngbr_8_ct < 5 and ddp_ngbr_8_ct > 2 order by ddp_ngbr_8_ct desc")
#sqldf("select distinct AgencyName from 'detBlightViolVD3C' where ddp_ngbr_8_ct < 5 and ddp_ngbr_8_ct > 2 order by ddp_ngbr_8_ct desc")
# Not Demolished --257958
# Demolished --     49846
#307804

count(*)
49846


In [69]:
indexes = sample(nrow(detBlightViolVD3C), size=0.4*nrow(detBlightViolVD3C))
test = detBlightViolVD3C[indexes,]
train = detBlightViolVD3C[-indexes,]


In [76]:
fol <- formula(Demolished ~ dbv_ngbr_8_ct + ddp_ngbr_8_ct + d311_ngbr_8_ct + dcr_ngbr_8_ct + 
               ViolationCategory + CleanUpCost + JudgmentAmt + PaymentStatus + FineAmt + AdminFee + 
               LateFee + AgencyName)

In [79]:
library(rpart)
rpModel <- rpart(fol, method="class", data=train)
rpp <- predict(rpartModel)
table(rpp, train$Demolished)

ERROR: Error in table(rpp, train$Demolished): all arguments must have the same length


In [78]:
library(randomForest)
rfModel <- randomForest(fol, data=train)
rfp <- predict(rfModel)
table(rfp, train$Demolished)

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:dplyr':

    combine

Warning message in randomForest.default(m, y, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"Warning message in data.matrix(x):
"NAs introduced by coercion"

ERROR: Error in randomForest.default(m, y, ...): NA/NaN/Inf in foreign function call (arg 1)
